<a href="https://colab.research.google.com/github/harika140901/EE5333_tutorials/blob/master/hierarchical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from typing import DefaultDict
import random
class Node:
  def __init__(self, name, area):
    self._name = name
    self._area = area
    self._ngbrs = []
    self._f = 0
    self._t = 0
    self._g = 0
    self._part = -1
  def __repr__(self):
    return self._name + f" ({round(self._area,2)})"

def updateNgbrs(V, E):
  for e in E:
    V[E[e][0][0]._name]._ngbrs.append(V[E[e][0][1]._name])
    V[E[e][0][1]._name]._ngbrs.append(V[E[e][0][0]._name])
  return V

def updateFTG(V_temp, E):
  for e in E:
    if E[e][0][0]._part != E[e][0][1]._part:
      V_temp[E[e][0][0]._name]._f += E[e][1]
      V_temp[E[e][0][1]._name]._f += E[e][1]
    else:
      V_temp[E[e][0][0]._name]._t += E[e][1]
      V_temp[E[e][0][1]._name]._t += E[e][1]
  for v in V_temp:
    V_temp[v]._g = V_temp[v]._f - V_temp[v]._t
  return V_temp

def area(V_temp):
  area = 0
  for v in V_temp:
    if V_temp[v]._part == 0:
      area += V_temp[v]._area
  return area

def partitionFM(V_temp, E_temp, Amin, Amax): #v and e temp here are dict of Nodes
  for v in V_temp:
    V_temp[v]._part = random.choice([0,1])
  V_temp = updateFTG(V_temp, E_temp)
  nodeList = []
  prevNodeListLen = -1
  while len(nodeList) < len(V_temp) and len(nodeList) != prevNodeListLen:
    maxg = []
    for v in V_temp:
      maxg.append(V_temp[v]._g)

    maxg = list(set(maxg))
    maxg.sort(reverse=True)
    for n in nodeList:
      if n._g in maxg:
        maxg.remove(n._g)
    prevNodeListLen = len(nodeList)
    for m in maxg:
      booltoexit = False
      for v in V_temp:
        if V_temp[v]._g == m and (V_temp[v]._part == 1 and area(V_temp) + V_temp[v]._area <= Amax) and v not in nodeList:
          nodeList.append(V_temp[v])
          V_temp[v]._part = 0
          booltoexit = True
          break
        elif V_temp[v]._g == m and (V_temp[v]._part == 0 and area(V_temp) - V_temp[v]._area >= Amin) and v not in nodeList:
          nodeList.append(V_temp[v])
          V_temp[v]._part = 1
          booltoexit = True
          break
      if booltoexit:
          break
    V_temp = updateFTG(V_temp, E_temp)
  return V_temp

class Node:
  def __init__(self, name, area):
    self._name = name
    self._area = area
    self._ngbrs = []
    self._f = 0
    self._t = 0
    self._g = 0
    self._sv = -1
    self._part = -1
  def __repr__(self):
    return self._name + f" ({round(self._area,2)})"

def updateEdgesWithVtempnodes(Edges,V_temp):
  Edgesnew = DefaultDict(list)
  for i in Edges:
    Edgesnew[str(i)] = ([V_temp[j._name] for j in Edges[i][0]], Edges[i][1])
  return Edgesnew

# N is the number of vertices; vertices: {0,1,2,...,N-1}
# E is a list of pairs of indices; e.g. [(0,4), (2,3) ... ]
# c is the coarsening parameter
def hier_part(N, E, c,s):
  V = {str(i): Node(str(i),1) for i in range(N)}
  Edges = DefaultDict(list)
  for i,e in enumerate(E):
    for j in e:
      Edges[str(i)] = ([Node(str(j),1) for j in e], 1)

  V_temp = updateNgbrs(V,Edges)
  Edges = updateEdgesWithVtempnodes(Edges, V_temp)

  V_temp_sv = V_temp
  Amax = (N//2) + (N//2)*s
  Amin = (N//2) - (N//2)*s
  svListForUncoarsening = []
  edgeListForUncoarsening = []
  edgeListForUncoarsening.append(Edges)
  for iter in range(c):
    ##############coarsing###############
    Nn = len(V_temp_sv)
    numOfNgbrs = [0] * Nn
    for i,n in enumerate(V_temp_sv):
      for e in Edges:
        if V_temp_sv[n] in Edges[e][0]:
          numOfNgbrs[i] += 1
    temp = []
    for i,n in enumerate(numOfNgbrs):
      temp.append([i,n])
    temp.sort(key=lambda x:x[1], reverse=True)
  #making supervetices
    listOfSVclubbed = []
    svIndex = 0
    superNodeList = DefaultDict(list)
    while len(listOfSVclubbed) < len(V_temp_sv):
      if temp[0][0] not in listOfSVclubbed:
        area = V_temp_sv[str(temp[0][0])]._area
        for j in V_temp_sv[str(temp[0][0])]._ngbrs:   #j = node
            if int(j._name) not in listOfSVclubbed and area + V_temp_sv[str(j._name)]._area <= Amax:
                j._sv = svIndex
                listOfSVclubbed.append(int(j._name))
                superNodeList[svIndex].append(V_temp_sv[str(j._name)])
        V_temp_sv[str(temp[0][0])]._sv = svIndex
        superNodeList[svIndex].append(V_temp_sv[str(temp[0][0])])
        listOfSVclubbed.append(int(temp[0][0]))
        svIndex += 1
      #remove listOfSVclubbed from temp
      for t in temp:
        if t[0] in listOfSVclubbed:
          temp.remove(t)

    #new list with supervertices and updated areas for each
    V_temp_sv_new = {str(i): Node(str(i),sum(x._area for x in superNodeList[i])) for i in superNodeList}

    #updating sv value in edge's nodes
    Edges = updateEdgesWithVtempnodes(Edges, V_temp_sv)

    # #updating sv in original V_temp
    # for v in V_temp:
    #   V_temp[v]._sv[iter] = V_temp_sv[v]
    EdgesforSVnew = DefaultDict(list)
    #making edges for supervertices and updating weight
    for e in Edges:
      if Edges[e][0][0]._sv != Edges[e][0][1]._sv :       # assuming no hyperedges
        if Edges[e][0][0]._sv > Edges[e][0][1]._sv:
          if (Edges[e][0][1]._sv,Edges[e][0][0]._sv) not in EdgesforSVnew.keys():
            EdgesforSVnew[(Edges[e][0][1]._sv,Edges[e][0][0]._sv)] = Edges[e][1]         #format of EdgesforSVnew = {(sv1,sv2):wt}
          else:
            EdgesforSVnew[(Edges[e][0][1]._sv,Edges[e][0][0]._sv)] += Edges[e][1]
        if Edges[e][0][0]._sv < Edges[e][0][1]._sv:
          if (Edges[e][0][0]._sv,Edges[e][0][1]._sv) not in EdgesforSVnew.keys():
            EdgesforSVnew[(Edges[e][0][0]._sv,Edges[e][0][1]._sv)] = Edges[e][1]
          else:
            EdgesforSVnew[(Edges[e][0][0]._sv,Edges[e][0][1]._sv)] += Edges[e][1]

    #changing EdgesforSV in {(sv2,sv2):([list of nodes],wt)}
    for e in EdgesforSVnew:
      EdgesforSVnew[e] = ([V_temp_sv_new[str(i)] for i in e],EdgesforSVnew[e])

    Edges = EdgesforSVnew
    svListForUncoarsening.append(V_temp_sv)
    V_temp_sv = updateNgbrs(V_temp_sv_new, EdgesforSVnew)
    edgeListForUncoarsening.append(EdgesforSVnew)

  #############coarsing over######################################
  svListForUncoarsening.append(V_temp_sv)
  p = 1
  while p <=(c + 1):
    EdgesforSVnew = edgeListForUncoarsening[-1*p]
    V_temp_sv = svListForUncoarsening[-1*p]
    V = partitionFM(V_temp_sv, EdgesforSVnew, Amin, Amax)
    print(V)
        #uncoarsening
    if p+1 <= len(svListForUncoarsening) :
        oldV = svListForUncoarsening[-1*(p+1)]
        for v in oldV:
            for g in V:
                if oldV[v]._sv == int(V[g]._name):
                    oldV[v]._part = V[g]._part
                    break
        V_temp_sv = oldV
    p += 1
  ########apply FM
  sol = list()
  for v in V_temp_sv:
    sol.append(V_temp_sv[v]._part)
  return sol

def cut_count(N, E, sol):
  count = 0
  assert len(sol) == N, 'invalid sol size!'
  for e in E:
    if sol[e[0]] != sol[e[1]]:
      count += 1
  return count

def rand_graph(N, p = 0.05):
  import networkx as nx
  import time
  g = nx.erdos_renyi_graph(N, 0.4)
  return g.edges

def test_hier_part():
  for N, c in [(100,1), (1000,2)]:
    E = rand_graph(N)
    print(len(E))
    sol = hier_part(N, E, c, 0.1)
    cut = cut_count(N, E, sol)
    print(f'N : {N}, c: {c} cut : {cut}')

test_hier_part()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-42-9db4e42f4573>", line 233, in <cell line: 0>
    test_hier_part()
  File "<ipython-input-42-9db4e42f4573>", line 227, in test_hier_part
    sol = hier_part(N, E, c, 0.1)
          ^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-42-9db4e42f4573>", line 193, in hier_part
    V = partitionFM(V_temp_sv, EdgesforSVnew, Amin, Amax)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-42-9db4e42f4573>", line None, in partitionFM
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
          ^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'Keyb